In [34]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats

## Задача про статзначимость изменений разных квантилей

In [47]:
# изменился ли статзначимо X-й квантиль?

# 1. вычисляем выборочную разность квантилей
# 2. считаем стандартное отклонение разности выборочных средних: 
#     - генерим много выборок из теста и контроля
#     - сохраняем их в список - это эмпирическое распределение
#     - потом считаем сигму
# 3. считаем половину смещения
# 4. считаем левый и правый интервал
# 5. проверяем, совпадают ли знаки выборочной разности и правого / левого интервала
    

In [26]:
logs = pd.read_csv('2022-04-13T12_df_web_logs.csv')
users = pd.read_csv('experiment_users.csv')

logs = logs.merge(users, how='inner', on='user_id')
logs = logs[
    (pd.to_datetime(logs['date']) >= datetime(2022, 4, 5))
        &
    (pd.to_datetime(logs['date']) < datetime(2022, 4, 12))
]


In [27]:
test_logs = logs[logs['pilot'] == 1].copy()['load_time']
control_logs = logs[logs['pilot'] == 0].copy()['load_time']


In [43]:
alpha = 0.05
quantiles = [
    0.7, 0.74, 0.78, 0.82, 0.86, 0.90, 0.95, 0.99, 0.999, 0.9999
]

In [46]:
for quantile_size in quantiles:
    bootstrap_calcs = []
    quantiles_delta = np.quantile(control_logs, quantile_size) - np.quantile(test_logs, quantile_size)
    
    for iter in range(5000):
        test_sample = np.random.choice(test_logs, size=len(test_logs), replace=True)
        ctrl_sample = np.random.choice(control_logs, size=len(control_logs), replace=True)
        bootstrap_calcs.append(np.quantile(ctrl_sample, quantile_size) - np.quantile(test_sample, quantile_size))
    
    bootstrap_sigma = np.std(bootstrap_calcs)
    sigma_shift = stats.norm.ppf(1 - alpha / 2)
    left_boarder = quantiles_delta - sigma_shift * bootstrap_sigma
    right_boarder = quantiles_delta + sigma_shift * bootstrap_sigma
#     print('quantile_size = ', quantile_size, ': ', left_boarder, quantiles_delta, right_boarder)
    if np.sign(left_boarder) == np.sign(quantiles_delta) and np.sign(right_boarder) == np.sign(quantiles_delta):
        print('quantile_size = ', quantile_size, ': yes')
    else:
        print('quantile_size = ', quantile_size, ': no')

quantile_size =  0.7 : yes
quantile_size =  0.74 : yes
quantile_size =  0.78 : no
quantile_size =  0.82 : yes
quantile_size =  0.86 : yes
quantile_size =  0.9 : yes
quantile_size =  0.95 : yes
quantile_size =  0.99 : yes
quantile_size =  0.999 : no
quantile_size =  0.9999 : no


## Задача про оценку эффекта бутстрапом

In [48]:
def generate_bootstrap_metrics(data_one, data_two, bootstrap_iter, bootstrap_agg_func):
    """Генерирует значения метрики, полученные с помощью бутстрепа.

    : data_one = значения метрик в контрольной группе.
    : data_two = значения метрик в тестовой группе.
    : design (Design)          = объект с данными, описывающий параметры эксперимента
    : bootstrap_iter (int)     = количество итераций бутстрепа.
    : bootstrap_agg_func (str) = метрика эксперимента. Возможные значения ['mean', 'quantile 95'].
    
    :return 
        bootstrap_metrics (np.array) - множество значений статистики теста, посчитанные на бутстрепных выборках.
        pe_metric (float)            - значение статистики теста посчитанное по исходным данным.
    """
    
    bootstrap_data_one = np.random.choice(data_one, (len(data_one), bootstrap_iter))
    bootstrap_data_two = np.random.choice(data_two, (len(data_two), bootstrap_iter))
    if bootstrap_agg_func == 'mean':
        bootstrap_metrics = (
            bootstrap_data_two.mean(axis=0) - bootstrap_data_one.mean(axis=0)
        )
        pe_metric = data_two.mean() - data_one.mean()
        return bootstrap_metrics, pe_metric
    elif bootstrap_agg_func == 'quantile 95':
        q = 0.95
        bootstrap_metrics = (
            np.quantile(bootstrap_data_two, q, axis=0)
            - np.quantile(bootstrap_data_one, q, axis=0)
        )
        pe_metric = np.quantile(data_two, q) - np.quantile(data_one, q)
        return bootstrap_metrics, pe_metric
    else:
        raise ValueError('Неверное значение bootstrap_agg_func')
        
data_one, data_two = np.array([1, 3]), np.array([5, 7])
bootstrap_iter = 10
bootstrap_agg_func = 'mean'

bootstrap_metrics, pe_metric = generate_bootstrap_metrics(
    data_one, data_two, bootstrap_iter, bootstrap_agg_func
)

print(bootstrap_metrics)
print(pe_metric)
# bootstrap_metrics = np.array([6., 5., 3., 4., 5., 2., 6., 4., 4., 4.])
# pe_metric = 4.0

[3. 6. 5. 3. 3. 4. 3. 4. 3. 3.]
4.0


In [53]:
import numpy as np
from scipy import stats


def run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type):
    """Строит доверительный интервал и проверяет значимость отличий с помощью бутстрепа.
    
    : bootstrap_metrics (np.array) = множество значений статистики теста, посчитанные на бутстрепных выборках.
    : param pe_metric (float): значение статистики теста посчитанное по исходным данным.
    : param alpha (float): уровень значимости.
    : param bootstrap_ci_type (str): способ построения доверительного интервала.
        Возможные значения ['normal', 'percentile', 'pivotal'].
        
    :return ci, pvalue:
        ci [float, float] - границы доверительного интервала.
        pvalue (float) - 0 если есть статистически значимые отличия, иначе 1. Настоящее
        pvalue для произвольного способа построения доверительного интервала с помощью
        бутстрепа вычислить не тривиально. Будем использовать краевые значения 0 и 1.
    """
    
    ci = []
    pvalue = 1
     
    if bootstrap_ci_type == 'normal':
        # строю нормальный доверительный интервал
        
        # смещение вправо (влево) в сигмах стандартизованного норм. распределения
        c = stats.norm.ppf(1 - alpha / 2)
        print('c = ', c)

        # стандартное отклонение выборки, сгенерированной бутстрапом
        se = np.std(bootstrap_metrics)
        print('se = ', se)

        # pe_metric = типа разность выборочных средних
        left, right = pe_metric - c * se, pe_metric + c * se
        ci.append(left)
        ci.append(right)
        
        if np.sign(left) == np.sign(pe_metric) and np.sign(right) == np.sign(pe_metric):
            pvalue = 0 
        else:
            pvalue = 1
        
        
    elif bootstrap_ci_type == 'percentile':
        # строю перцентильный доверительный интервал
        
        left, right = np.quantile(bootstrap_metrics, [alpha / 2, 1 - alpha / 2])
        ci.append(left)
        ci.append(right)
        
        if np.sign(left) == np.sign(pe_metric) and np.sign(right) == np.sign(pe_metric):
            pvalue = 0 
        else:
            pvalue = 1
    
    elif bootstrap_ci_type == 'pivotal':
        # строю центральный доверительный интервал
        
        right, left = 2 * pe_metric - np.quantile(bootstrap_metrics, [alpha / 2, 1 - alpha / 2])
        ci.append(left)
        ci.append(right)
        
        if np.sign(left) == np.sign(pe_metric) and np.sign(right) == np.sign(pe_metric):
            pvalue = 0 
        else:
            pvalue = 1
    
    return ci, pvalue

In [54]:
bootstrap_metrics = np.arange(-90, 910)
pe_metric = 600.
alpha = 0.05
bootstrap_ci_types = ['normal', 'percentile', 'pivotal']

for bootstrap_ci_type in bootstrap_ci_types:
    ci, pvalue = run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type)
    print(bootstrap_ci_type)
    print(f'ci = {np.array(ci).round()}, pvalue = {pvalue}')
    
# >>> normal: ci = [  34. 1166.], pvalue = 0.0
# >>> percentile: ci = [-65. 884.], pvalue = 1.0
# >>> pivotal: ci = [ 316. 1265.], pvalue = 0.0

c =  1.959963984540054
se =  288.6749902572095
normal
ci = [  34. 1166.], pvalue = 0
percentile
ci = [-65. 884.], pvalue = 1
pivotal
ci = [ 316. 1265.], pvalue = 0
